In [4]:
from competition_toolkit.dataloader import download_dataset
from torch.utils.data import Dataset

# Load dataset

In [6]:
trainDataset = download_dataset("train", 1, get_dataset=True)
testDataset = download_dataset("validation", 1, get_dataset=True)

print(trainDataset)
print(f'Sample: {trainDataset[0]}')
print(testDataset)
print(f'Sample: {testDataset[0]}')

Found cached dataset mapai_training_data (/home/lixado/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)
Found cached dataset mapai_training_data (/home/lixado/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)


Dataset({
    features: ['image', 'lidar', 'mask'],
    num_rows: 7000
})
Sample: {'image': '../../data/train/images/6179_495_44.tif', 'lidar': '../../data/train/lidar/6179_495_44.tif', 'mask': '../../data/train/masks/6179_495_44.tif'}
Dataset({
    features: ['image', 'lidar', 'mask'],
    num_rows: 1500
})
Sample: {'image': '../../data/validation/images/6305_471_68.tif', 'lidar': '../../data/validation/lidar/6305_471_68.tif', 'mask': '../../data/validation/masks/6305_471_68.tif'}
